<div>
<img src="figures/svtLogo.png"/>
</div>

<h1><center>Mathematical Optimization for Engineers</center></h1>
<h2><center>Bonus exercise 3 </center></h2>

## Background
### Optimal control problems

$$\newcommand{\R}{\mathbb R}$$
In this programming exercise, you will solve an optimal control problem (OCP). OCPs are a special case of dynamic optimization problems. For this exercise, we introduce the following class of optimal control problems:

\begin{align}
	\min_{\mathbf{x}(\cdot),\mathbf{u}(\cdot)}  & \Phi(\mathbf{x}(\cdot)) = \mathbf{\phi}(\mathbf{x}(t_f)) \\
	\mbox{s.t. }\quad & \dot{\mathbf{x}}(t) = \mathbf{f}(\mathbf{x}(t),\mathbf{u}(t)), \; t \in [t_0,t_f], \\
    &\mathbf{x}(t_0) = \mathbf{x_0}, \\
	&\mathbf{u}_{\min}  \leq \mathbf{u}(t) \leq \mathbf{u}_{\max} \; \forall \; t \in [t_0,t_f].
\end{align}
    
The state variables $\mathbf{x}(t) \in \R^{n_x}$ and control variables $\mathbf{u}(t) \in \R^{n_u}$ are time-dependent. The so-called Mayer-type objective functional $\Phi$ is defined by the function $\mathbf{\phi} : \R^{n_x} \rightarrow \R$, that only depends on the state $\mathbf{x}(t)$ at the final time $t_f$. The constraints are a system of ordinary differential equations (ODE). The right hand side of the ODE is given by the function $\mathbf{f}:\R^{n_x}\times \R^{n_u} \rightarrow \R^{n_x}$. Finally, we consider control path constraints in form of simple lower and upper bounds $\mathbf{u}_{\min} \in \R^{n_u}$ and $\mathbf{u}_{\max} \in \R^{n_u}$ on $\mathbf{u}(t)$, respectively.

The dimension of the optimization problem is infinite, since for every $t\in [t_0,t_f]$, $\mathbf{u}(t)$ and $\mathbf{x}(t)$ are optimization variables.

For fixed $\mathbf{u}(\cdot)$, the state variables $\mathbf{x}(t),\,t\in [t_0,t_f]$ are uniquely determined by the solution of the initial value problem. Thus, the control vector function $\mathbf{u}:[t_0,t_f] \rightarrow \R^{n_u}$  is the actual (infinite-dimensional) degree of freedom.

### Full discretization approach

$$\newcommand{\R}{\mathbb R}$$
The so-called full discretization approach discretizes state and control variables, as well as the differential equations. Thus, the original optimal control problem is transformed into an ordinary nonlinear program (NLP). Here, we will use the <u>implicit Euler method</u> to discretize the ODE into a set of nonlinear equations. The procedure to obtain a nonlinear program is now described in detail:

The first step is to divide the time horizon $[t_0,t_f]$ into $M$ intervals $[t_{k-1},t_k]$, $k=1,\dots, M$ of length $h$ with

$
t_M = t_f, \qquad t_k - t_{k-1} = h, \, k=1,\dots, M, \qquad h = \frac{t_f-t_0}{M}.
$

The implicit Euler discretization is then
\begin{equation}\label{eq:euler}
\mathbf{x}_{k+1} = \mathbf{x}_k + h \cdot \mathbf{f}(\mathbf{x}_{k+1},\mathbf{u}_{k+1}), \quad k=0,1,\dots,M-1,
\end{equation}

where $\mathbf{x}_{k}\in \R^{n_x}$ and $\mathbf{u}_k\in \R^{n_u}$ are finite dimensional decision variables that approximate the states $\mathbf{x}(t)$ and controls $\mathbf{u}(t)$, respectively, at the discretization points
$t_k$, $k=1,2,\dots,M$.

$$\newcommand{\R}{\mathbb R}$$
The optimization variable vector of the full discretization NLP is 

\begin{align}
\mathbf{y} = \left(\begin{array}{c}
\mathbf{x}_1\\
\mathbf{u}_1\\
\mathbf{x}_2\\
\mathbf{u}_2\\
\vdots \\
\mathbf{x}_M\\
\mathbf{u}_M
\end{array} \right) \in \R^{n_y}, \quad \text{where } n_y = M\cdot(n_x+n_u).
\end{align}

The full discretization NLP is

\begin{align}
\min_{\mathbf{y} \in \R^{n_y}} & \phi(\mathbf{x}_M) \\
\text{s.t.}\quad & \mathbf{c}_k(y) = \mathbf{0}, \quad k=0,1,\dots, M-1 \\
& \mathbf{u}_{\min} \le \mathbf{u}_k \le \mathbf{u}_{\max}, \quad k=1,\dots,M,
\end{align}

where the constraint functions $\mathbf{c}_k: \R^{n_y} \rightarrow \R^{n_x}$, $k=0,1,\dots,M-1$ are defined by:
$
\mathbf{c}_k(\mathbf{y}) := \mathbf{x}_{k+1} - \mathbf{x}_k - h \cdot \mathbf{f}(\mathbf{x}_{k+1},\mathbf{u}_{k+1}).
$

## Problem description

We consider reactions inside a cylindrical fixed bed reactor:
<br>
<br>
<div>
<img src="figures/fixedBed.png" width="500"/>
</div>


For modelling purposes, think of it as a cylndrical tube, filled with catalyst pellets (bed), with reactants flowing through the bed and being converted into products. The reactant enters the reactor on the inflow side, reacts inside the reactor and leaves together with the products on the outflow side.
<br>
<br>
<div>
<img src="figures/TubularReactor-001.png" width="400"/>
</div>

We additionally assume the fluid velocity inside the cylinder to be uniform over a cross-section $A_c$, the flow to be in steady-state and the density of the fluid to be constant.

In our problem, the reactions inside the reactor can be described by the following chemical equation:
<br>
<br>
<div>
<img src="figures/catReaction.png" width="150"/>
</div>

The reactions are catalysed - the reversible step from species $A$ to intermediate species $B$, $A  \rightleftharpoons B$, is dependent on catalyst 1 and the irreversible step from $B \rightarrow C$ is dependent on catalyst 2. 

The objective is to maximize the amount of $C$ at the outflow by adjusting the mixture of catalysts at all positions $t$ on the bed. The control variable, $u(t)$, is the fraction of catalyst 1 and consequently, $(1-u(t))$ is the fraction of catalyst 2 on the cross-section of the bed, at position $t$. 

It is important to remark, that the variable $t$ does not represent the time in this problem, but the spatial dimension in direction of the reactor axis.

To solve the problem using full-discretization, we first discretize the 1-dimensional space as shown:
<br>
<br>
<div>
<img src="figures/TubularReactor-002.png" width="400"/>
</div>
<br>
<br>
The equations below show the mathematical formulation of this setting:<br> 
<i>(Hint: The flow is assumed uniform and steady, so the space and time derivatives are interchangeable with the help of a constant. You can assume the constant flow speed to be $1m/s$ and the length of the reactor, $t_f = 1m$)</i>:

\begin{align}
\begin{split}
\frac{d x_A(t)}{d t} &= u(t) \cdot (k_2x_B(t) - k_1x_A(t))\\
\\
\frac{d x_B(t)}{d t} &= -u(t) \cdot (k_2x_B(t)-k_1x_A(t))-(1-u(t))\cdot k_3 x_B(t)\\
\\
x_C(t) &= 1-x_A(t)-x_B(t), \; \; t \in [t_0,t_f] \\
\\
u_{min} \; &\le \; u(t) \; \le \; u_{max} \; \forall \; t \in [t_0,t_f]\\
\\
\mathbf{\phi}(x_A(t_f)&, x_B (t_f))=x_C(t_f) = 1-x_A(t_f)-x_B(t_f)
\end{split}
\end{align}
<br>
<br>
$x_A(t)$ and $x_B(t)$ are the unknown mole fractions of the educts $A$ and $B$ at position $t$, where $t$ is the distance from the inflow.

$k_1$, $k_2$ and $k_3$ are the velocity constants of reactions $1$, $2$ and $3$. In the following, the values $k_1=k_3=1$, $k_2=10$, $t_0=0$ and $t_f=1$ shall be used. 

As stated before the control variable $u$ represents the mole fraction of  catalyst 1, therefore $u_{min}=0$ and $u_{max}=1$.

The inflow only consists of reactant $A$. So the last piece of information needed to solve this problem are the initial conditions:
\begin{align}
\begin{split}
{x}_A(t_0) &= 1\\
{x}_B(t_0) &= 0.\label{initialvalue}
\end{split}
\end{align}

The goal is to maximize the mole fraction of product $C$, namely 
$
x_C(t_f) = \phi(x_A(t_f), x_B (t_f)) =
1-x_A(t_f)-x_B(t_f)
$
at the final position ($t_f = 1$).

### Your task

1. Set up the nonlinear program by applying full discretization.
<br>
<br>
2. Solve the nonlinear optimization problem using the solver SLSQP. Use the value $0.5$ as initial guess for $u_k$, and $(1,0)^T$ (why?) as an initial guess for $x_k$.

#### Hint

The computed optimal objective function values for $M=10$, $M=30$ and $M=50$ are
0.0450, 0.0468 and 0.0472, respectively.

<u>You should use the template below, for submission.</u> 
<br>
<br>
- As a rule of thumb, you should <b>only add code</b> to this file and <b>not delete or change anything in the template</b>.
<br>
<br>
- To see where you can add code, go ahead and do a search (ctrl+f) for '# YOUR CODE HERE'
<br>
<br>
- Once you have added your code at these spots, delete the subsequent 'raise NotImplementedError()'. These exist only to remind you to add your own code.
<br>
<br>
- Most importantly, have fun while programing :)!

In [1]:
import numpy as np

# refer to the documentation of scipy's SLSQP solver to understand these imports
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import NonlinearConstraint

# don't worry about plotting
from matplotlib import pyplot as plt

In [2]:
# Right hand side function of ODE (10 points)

# inputs: state and control vectors, x and u, as numpy arrays

# output: RHS vector, f, as numpy array

def eval_rhs(x, u):
    # constants
    k1 = 1
    k2 = 10
    k3 = 1
    
    f = np.zeros(2)
    
    # YOUR CODE HERE
    f[0] = u*(k2*x[1]-k1*x[0])
    f[1] = (-1*u*(k2*x[1]-k1*x[0]))-((1-u)*k3*x[1])
    #raise NotImplementedError()
    
    return f

In [3]:
# please leave this cell as it is

In [4]:
# equality constraints for optimization - discretized ODE (50 points)

# inputs: 
#  1. optimization variable vector of the full-discretization NLP, y, as numpy array
#  2. number of discretization intervals, M, as integer

# output: 
#  1. vector of evaluated contraint expressions, c_k(y) for all k = 0,1 ... ,M-1, as numpy array
#
#  Note - Please ensure that the output is in the format (c_0', c_1', c_2', ... , c_M-1')' i.e. all column vectors 
#         c_k(y) are vertically collated into one big column vector, ceq, in the order shown.

def cons(y, M):
    
    # YOUR CODE HERE
    t0 = 0
    tf = 1
    h = (tf-t0)/M
    nx = 2
    nu = 1
    
    ceq = np.zeros(nx*M)
    x0 = np.zeros(2)
    x0[0] = 1
    x0[1] = 0
    
    fval = eval_rhs(y[0:nx], y[nx:nx+nu])
    ceq[0:nx] = x0 + (h*fval) - y[0:nx]
    
    for i in np.arange(2, M+1):  
        c0i = (i-1) * nx
        c1i = [int(a) for a in range(c0i, c0i+nx)] 
        
        xai = (i-1)*(nx+nu)
        xbi = [int(a) for a in range(xai, xai+nx)] 
        xi = [int(a) for a in range(xai-nx-nu, xai-nu)]
        ui = int(xai+nx)
        
        fval = eval_rhs(y[xbi],y[ui])
        ceq[c1i] = y[xi]+(h*fval)-y[xbi]
         
    return ceq

In [5]:
# please leave this cell as it is

In [6]:
# objective function (10 points)

# inputs: 
#  1. optimization variable vector of the full-discretization NLP, y, as numpy array

# output: 
#  1. objective function value 

def objective(y):

    # YOUR CODE HERE
    obj = y[-2]+y[-3]-1
    #raise NotImplementedError()
    return obj

In [7]:
# please leave this cell as it is

In [8]:
# main function (30 points)

# inputs: 
#  1. number of discretization intervals, M, as integer

# output: 
#  1. optimal objective function value 
#  2. optimal variable vector, y, as numpy array

def run_optimization(M):

    # YOUR CODE HERE
    nx = 2
    nu = 1
    len_y = (nx+nu)*M
    
    ui = (np.arange(2,len_y,3)).astype(int)
    x1i = [int(a) - 2 for a in ui]
    x2i = [int(a) - 1 for a in ui]
    
    y0 = np.zeros((len_y))
    y0[x1i] = 1
    y0[x2i] = 0
    y0[ui] = 0.5
    
    lb = np.zeros(len_y)
    ub = np.ones((len_y))    
    bounds_u = Bounds(lb, ub)
    
    cons_ini = lambda y: cons(y, M)
    nl_cons = NonlinearConstraint(cons_ini, 0, 0)
    
    res = minimize(objective, y0, method='SLSQP', constraints=[nl_cons], bounds=bounds_u)
    objopt = -1*res.fun
    yopt = res.x
    
    return objopt, yopt
print("Done!")

Done!


In [9]:
objopt, yopt = run_optimization(10)
print("objopt: ", objopt)
print("yopt: ", yopt)

objopt:  0.04504496717523454
yopt:  [0.95494276 0.04454189 0.88430076 0.93686674 0.05982918 0.53388539
 0.92705673 0.06544991 0.35992446 0.92019861 0.06744572 0.27907859
 0.91450717 0.06798757 0.24256934 0.90939335 0.06781953 0.22118781
 0.90482684 0.06698922 0.19437387 0.90128118 0.06495431 0.14084718
 0.90043782 0.05996894 0.02804195 0.90043782 0.05451722 0.        ]


In [10]:
objopt, yopt = run_optimization(30)
print("objopt: ", objopt)
print("yopt: ", yopt)

objopt:  0.04679239222960985
yopt:  [9.75609756e-01 2.43902439e-02 1.00000000e+00 9.57763236e-01
 4.22367638e-02 1.00000000e+00 9.46537213e-01 5.31281131e-02
 8.11018849e-01 9.39076278e-01 5.99067100e-02 6.58299898e-01
 9.33807585e-01 6.41944210e-02 5.41557530e-01 9.29870918e-01
 6.69108639e-02 4.52903026e-01 9.26771873e-01 6.86063154e-02
 3.86239969e-01 9.24213097e-01 6.96257740e-02 3.36746772e-01
 9.22008381e-01 7.01939138e-02 3.00548214e-01 9.20037133e-01
 7.04612117e-02 2.74515278e-01 9.18219477e-01 7.05299690e-02
 2.56104311e-01 9.16502168e-01 7.04696550e-02 2.43238614e-01
 9.14850264e-01 7.03263999e-02 2.34216947e-01 9.13242098e-01
 7.01290266e-02 2.27621666e-01 9.11666030e-01 6.98931318e-02
 2.22257081e-01 9.10118359e-01 6.96238106e-02 2.17082407e-01
 9.08602053e-01 6.93174219e-02 2.11154037e-01 9.07126077e-01
 6.89626176e-02 2.03578446e-01 9.05705368e-01 6.85406412e-02
 1.93464498e-01 9.04361563e-01 6.80248156e-02 1.79876099e-01
 9.03124733e-01 6.73790393e-02 1.61786507e-01 9.0

In [11]:
objopt, yopt = run_optimization(50)
print("objopt: ", objopt)
print("yopt: ", yopt)

objopt:  0.047204348990063205
yopt:  [9.83606557e-01 1.63934426e-02 1.00000000e+00 9.70169309e-01
 2.98306907e-02 1.00000000e+00 9.59155172e-01 4.08448284e-02
 1.00000000e+00 9.51031394e-01 4.88495877e-02 8.78178749e-01
 9.44896593e-01 5.47343293e-02 7.71569634e-01 9.40143781e-01
 5.91098191e-02 6.80829653e-01 9.36371438e-01 6.23875321e-02
 6.03582440e-01 9.33307789e-01 6.48517923e-02 5.37878709e-01
 9.30764972e-01 6.67036806e-02 4.82090631e-01 9.28610577e-01
 6.80884502e-02 4.34834088e-01 9.26749549e-01 6.91131038e-02
 3.94923811e-01 9.25112441e-01 6.98579023e-02 3.61339197e-01
 9.23647672e-01 7.03840199e-02 3.33193329e-01 9.22316379e-01
 7.07387011e-02 3.09704626e-01 9.21088862e-01 7.09588753e-02
 2.90192717e-01 9.19942167e-01 7.10736655e-02 2.74059327e-01
 9.18858403e-01 7.11061597e-02 2.60774630e-01 9.17823556e-01
 7.10747015e-02 2.49870298e-01 9.16826634e-01 7.09938406e-02
 2.40932197e-01 9.15859065e-01 7.08750226e-02 2.33589321e-01
 9.14914221e-01 7.07271552e-02 2.27516505e-01 9.

In [12]:
# please leave this cell as it is